## 예술의전당 콘서트홀 가격 모델

### 1. 라이브러리

In [4]:
# base
import pandas as pd
import numpy as np
import re

# files
from glob import glob

# visualization
import matplotlib.pyplot as plt
import koreanize_matplotlib
import plotly.express as px
import seaborn as sns

In [5]:
# settings
%matplotlib inline
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)


### 2. 데이터셋 로드

In [177]:
def load_file (file_name, file_num=0):
    files = glob(file_name)
    file = files[file_num]   
    df = pd.read_csv(files[file_num], low_memory=False)

    print(f"Found.. {len(files)} file(s) : {files}")    
    print(f"Reading.. {files[file_num]}")
    print(f"dataframe.shape : {df.shape}")
    
    return df

In [179]:
# df : 예매데이터 + 좌석데이터 merge된 데이터셋
df = load_file('클래식_빈좌석포함데이터.csv')
df = df[(df['place']=='콘서트홀') & (df['genre']=='클래식')]
print(f"df.shape : {df.shape}")

Found.. 1 file(s) : ['클래식_빈좌석포함데이터.csv']
Reading.. 클래식_빈좌석포함데이터.csv
dataframe.shape : (405810, 42)
df.shape : (405810, 42)


### 3. 원가격추정-1 : 판매된 좌석의 할인 전 금액 추정하기

discount_type 에서 '할인율' 혹은 '할인액'을 구해서 'price'에서 역산한다.

In [181]:
# discount_type에 적용된 할인의 종류가 다양함.
df['discount_type'].value_counts()

초대권            93749
일반             22264
기획사판매          21029
골드회원 할인10%      6634
그린회원 할인5%       3307
               ...  
그린회원 할인30%         1
노블회원 할인50%         1
특판B 30%            1
기획사할인50%           1
골드회원 할인10%_        1
Name: discount_type, Length: 213, dtype: int64

In [182]:
# 할인율 계산 > % 앞의 숫자만 추출
df['할인율'] = df['discount_type'].str.extract('(\d+)%')
df['할인율'] = df['할인율'].fillna(0).astype(int)
df['할인율'] = df['할인율'] / 100
print(df['할인율'].value_counts().sort_index())

# NaN인 경우 (티켓이 판매되지 않은 경우) 채워넣기
df[['할인율', 'price']] = df[['할인율', 'price']].fillna(0)

0.00    368076
0.05      9230
0.10     10125
0.12       640
0.15       974
0.20      4371
0.25       372
0.30      5351
0.40      3597
0.50      3074
Name: 할인율, dtype: int64


In [183]:
# discount_type에 할인율이 명시되어 있지 않은 경우 > 대부분 초대권, 기획사판매, 예매권
df.loc[df['할인율']==0, 'discount_type'].value_counts()

초대권                  93749
일반                   22264
기획사판매                21029
공연진행석                 2406
홍보진행                  1131
당일할인티켓                 981
기획사할인                  695
차액                     339
중앙일보 JTBC              200
당일할인티켓_                116
중앙일보 JTBC 초대권          100
공연예매권                   88
기획사                     18
하비에르 국제학교 학부모, 직원       13
수험생 할인(동반1인)            10
하비에르 국제학교 재학생            9
싹틔우미 할인                  3
Name: discount_type, dtype: int64

In [184]:
# '할인전금액' 구하기 : 'price(판매금액)'에 '할인율'을 역산
df['할인전금액'] = df['price'] / (1 - df['할인율'])
df['할인전금액'] = df['할인전금액'].astype(int)

df[['discount_type', 'price', '할인율', '할인전금액']]

,discount_type,price,할인율,할인전금액
0,기획사판매,0.0,0.00,0
1,기획사판매,0.0,0.00,0
2,기획사판매,0.0,0.00,0
3,기획사판매,0.0,0.00,0
4,기획사판매,0.0,0.00,0
...,...,...,...,...
405805,장애인/국가유공자 할인50%,15000.0,0.50,30000
405806,장애인/국가유공자 할인50%,15000.0,0.50,30000
405807,그린회원 할인5%,28000.0,0.05,29473
405808,NaN,0.0,0.00,0


In [185]:
# 입장권이 5천원도 안되는 금액일 때 => 이상치 같은데 0원으로 처리?
df.loc[(df['할인전금액'] < 5000) & (df['할인전금액'] != 0), '할인전금액'] = 0

### 4. 원가격추정-2 : 판매되지 않은 좌석의 티켓값 구하기

In [240]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression



#### KNN (scoring='neg_mean_squared_error')

In [ ]:
공연별_df_list = []
booked_rate_list = []
best_k_list = []
mse_list = []

공연_list = sorted(df['전체공연시간'].unique())
for c in 공연_list:
    # 공연시간으로 특정해서 공연마다 별도로 본다.
    공연별_df = df[df['전체공연시간'] == c]

    # 예매된 경우 (가격을 알 수 있음), 예매되지 않은 경우 (가격을 모름)을 구분한다.
    booked_seat = 공연별_df[공연별_df['할인전금액'] > 0]
    emppty_seat = 공연별_df[공연별_df['할인전금액'] == 0].drop('할인전금액', axis=1)
    booked_rate = round(booked_seat.shape[0] / 공연별_df.shape[0], 3)
    booked_rate_list.append(booked_rate)

    # 가격을 알 수 있는 좌석이 하나도 없을 경우 continue
    if booked_rate == 0:
        best_k_list.append(np.nan)
        mse_list.append(np.nan)
        공연별_df_list.append(공연별_df)
        continue


    # == step1 : best_k (가장 적절한 n_neighbors)값 찾기 ==
    X = booked_seat[['X', 'Y', 'Z']]
    y = booked_seat['할인전금액']
    cv_scores = []
    for n in range(1, min(30, X.shape[0])*4//5 + 1):
        model = KNeighborsRegressor(n_neighbors=n, weights='distance', p=1)
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        cv_scores.append(scores.mean())
    best_k = [i for i in range(1, 51)][np.argmax(cv_scores)]

    best_k_list.append(best_k)
    mse_list.append(max(cv_scores) * -1)


    # == step2 : best_k 값 적용하기 ==
    model = KNeighborsRegressor(n_neighbors=best_k, weights='distance', p=1)
    model.fit(X, y)
    emppty_seat['할인전금액'] = np.round(model.predict(emppty_seat[['X', 'Y', 'Z']]), -2).astype(int)

    공연별_df = pd.concat([booked_seat, emppty_seat], axis=0).sort_values(by='seat')
    공연별_df = 공연별_df.reset_index().drop('index', axis=1)
    공연별_df_list.append(공연별_df)

#### KNN (scoring = default)

In [258]:
공연별_df_list = []
booked_rate_list = []
best_k_list = []
mse_list = []

공연_list = sorted(df['전체공연시간'].unique())
for c in 공연_list:
    # 공연시간으로 특정해서 공연마다 별도로 본다.
    공연별_df = df[df['전체공연시간'] == c]

    # 예매된 경우 (가격을 알 수 있음), 예매되지 않은 경우 (가격을 모름)을 구분한다.
    booked_seat = 공연별_df[공연별_df['할인전금액'] > 0]
    emppty_seat = 공연별_df[공연별_df['할인전금액'] == 0].drop('할인전금액', axis=1)
    booked_rate = round(booked_seat.shape[0] / 공연별_df.shape[0], 3)
    booked_rate_list.append(booked_rate)

    # 가격을 알 수 있는 좌석이 하나도 없을 경우 continue
    if booked_rate == 0:
        best_k_list.append(np.nan)
        mse_list.append(np.nan)
        공연별_df_list.append(공연별_df)
        continue


    # == step1 : best_k (가장 적절한 n_neighbors)값 찾기 ==
    X = booked_seat[['X', 'Y', 'Z']]
    y = booked_seat['할인전금액']

    # best_k 찾기
    cv_scores = []
    for n in range(1, min(30, X.shape[0])*4//5 + 1):
        model = KNeighborsRegressor(n_neighbors=n, weights='distance', p=2)
        scores = cross_val_score(model, X, y, cv=5)
        cv_scores.append(scores.mean())
    best_k = [i for i in range(1, 51)][np.argmax(cv_scores)]
    best_k_list.append(best_k)

    # mse값 구하기
    model = KNeighborsRegressor(n_neighbors=best_k, weights='distance', p=2)
    mse_score = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    mse_list.append(mse_score * -1)


    # == step2 : best_k 값 적용하기 ==
    model = KNeighborsRegressor(n_neighbors=best_k, weights='distance', p=2)
    model.fit(X, y)
    emppty_seat['할인전금액'] = np.round(model.predict(emppty_seat[['X', 'Y', 'Z']]), -2).astype(int)

    공연별_df = pd.concat([booked_seat, emppty_seat], axis=0).sort_values(by='seat')
    공연별_df = 공연별_df.reset_index().drop('index', axis=1)
    공연별_df_list.append(공연별_df)

#### LinearRegression

In [250]:
공연별_df_list = []
booked_rate_list = []
mse_list = []

공연_list = sorted(df['전체공연시간'].unique())
for c in 공연_list:
    # 공연시간으로 특정해서 공연마다 별도로 본다.
    공연별_df = df[df['전체공연시간'] == c]

    # 예매된 경우 (가격을 알 수 있음), 예매되지 않은 경우 (가격을 모름)을 구분한다.
    booked_seat = 공연별_df[공연별_df['할인전금액'] > 0]
    emppty_seat = 공연별_df[공연별_df['할인전금액'] == 0].drop('할인전금액', axis=1)
    booked_rate = round(booked_seat.shape[0] / 공연별_df.shape[0], 3)
    booked_rate_list.append(booked_rate)

    # 가격을 알 수 있는 좌석이 하나도 없을 경우 continue
    if booked_rate == 0:
        mse_list.append(np.nan)
        공연별_df_list.append(공연별_df)
        continue


    # == step1 : score ==
    X = booked_seat[['X', 'Y', 'Z']]
    y = booked_seat['할인전금액']
    model = LinearRegression()
    score = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    mse_list.append(-np.mean(score))


    # == step2 ==
    model.fit(X, y)
    emppty_seat['할인전금액'] = np.round(model.predict(emppty_seat[['X', 'Y', 'Z']]), -2).astype(int)

    공연별_df = pd.concat([booked_seat, emppty_seat], axis=0).sort_values(by='seat')
    공연별_df = 공연별_df.reset_index().drop('index', axis=1)
    공연별_df_list.append(공연별_df)

#### model별 결과

knn모델에서 scoring='neg_mean_squared_error'로 설정할 때 가장 성능이 좋음.  
여전히 mse가 높게 측정되는 문제 발생

In [259]:
knn_default_result = pd.DataFrame({'전체공연시간' : list(df['전체공연시간'].unique()),
                           '가격명시좌석비율' : booked_rate_list,
                           'best_k' : best_k_list,
                           'mse' : mse_list})

knn_default_result

,전체공연시간,가격명시좌석비율,best_k,mse
0,2018-11-25 17:00:00,0.019,4.0,7.399848e+08
1,2018-12-08 17:00:00,0.021,5.0,2.844223e+08
2,2019-01-11 20:00:00,0.319,10.0,6.044321e+08
3,2019-02-05 20:00:00,0.022,1.0,7.423864e+08
4,2019-02-16 17:00:00,0.147,1.0,6.325557e+07
...,...,...,...,...
157,2023-05-09 19:30:00,0.402,2.0,3.006422e+08
158,2023-05-21 17:00:00,0.303,23.0,4.844809e+08
159,2023-05-23 19:30:00,0.469,16.0,4.908790e+08
160,2023-06-01 19:30:00,0.315,8.0,8.449852e+08


In [257]:
lr_result.describe()

,가격명시좌석비율,mse
count,162.000000,1.510000e+02
mean,0.151525,6.881934e+08
std,0.134406,2.651119e+09
min,0.000000,-0.000000e+00
25%,0.045000,1.124932e+08
50%,0.115000,2.636463e+08
75%,0.239750,5.306039e+08
max,0.631000,3.213141e+10


In [239]:
knn_result_m.describe()

,가격명시좌석비율,best_k,mse
count,162.000000,151.000000,1.510000e+02
mean,0.151525,8.794702,5.178022e+08
std,0.134406,6.941967,6.396762e+08
min,0.000000,1.000000,2.581515e-24
25%,0.045000,3.000000,1.368201e+08
50%,0.115000,7.000000,3.170575e+08
75%,0.239750,13.000000,6.975956e+08
max,0.631000,24.000000,4.496407e+09


In [261]:
knn_default_result.describe()

,가격명시좌석비율,best_k,mse
count,162.000000,151.000000,1.510000e+02
mean,0.151525,7.642384,5.224467e+08
std,0.134406,6.652663,6.109527e+08
min,0.000000,1.000000,4.371783e-24
25%,0.045000,2.000000,1.427370e+08
50%,0.115000,6.000000,3.133432e+08
75%,0.239750,11.000000,7.287752e+08
max,0.631000,24.000000,3.887068e+09


In [236]:
knn_result.describe()

,가격명시좌석비율,best_k,mse
count,162.000000,151.000000,1.510000e+02
mean,0.151525,8.039735,5.078254e+08
std,0.134406,6.849701,6.029574e+08
min,0.000000,1.000000,4.371783e-24
25%,0.045000,2.000000,1.385371e+08
50%,0.115000,6.000000,3.061935e+08
75%,0.239750,12.000000,6.681405e+08
max,0.631000,24.000000,3.887068e+09


In [230]:
# 총 162개 공연 중
print(f"총 {df['전체공연시간'].nunique()} 공연 중..")
print(f"가격을 알 수 없는 공연 : {sum([1 for i in booked_rate_list if i == 0])}개")
print(f"가격을 알 수 있는 공연 : {sum([1 for i in booked_rate_list if i > 0])}개")

new_df = pd.concat(공연별_df_list, axis=0, ignore_index=True)
new_df.shape

총 162 공연 중..
가격을 알 수 없는 공연 : 11개
가격을 알 수 있는 공연 : 151개


(405810, 44)

In [231]:
new_df['할인전금액'].describe()

count    405810.000000
mean      71707.446514
std       47981.258918
min           0.000000
25%       40000.000000
50%       63900.000000
75%      100000.000000
max      320000.000000
Name: 할인전금액, dtype: float64

In [235]:
temp = list(공연별_df_list)[30]

fig = px.scatter_3d(temp, x='X', y='Y', z='Z', color='할인전금액', hover_name='seat')
fig.update_layout(width=800, height=600)
fig.update_traces(marker={'size': 1})
fig.show()

In [174]:
pd.Series(booked_rate_dict).describe()

count    162.000000
mean       0.151525
std        0.134406
min        0.000000
25%        0.045000
50%        0.115000
75%        0.239750
max        0.631000
dtype: float64